In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

!pip install pytorch_transformers

In [0]:
import pandas as pd
import numpy as np
import unicodedata
import torch

from pytorch_transformers import BertTokenizer, BertModel

In [0]:
import pickle
import gzip

with gzip.open('/content/gdrive/My Drive/pre_reviews.pickle', 'rb') as f:
    pre_reviews = pickle.load(f)

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
import pickle
import gzip

with gzip.open('/content/gdrive/My Drive/Top5000_BtoA.pickle','rb') as f:
    Top5000_BtoA = pickle.load(f)

In [0]:
# BERT
def preprocess(review: str, total: int, show_progress: bool = True) -> list:
    if show_progress:
        global counter
        counter += 1
        if counter % 10000 == 0:
          print('Processing... %6i/%6i'% (counter, total))
        
    token_ids = tokenizer.encode(review, add_special_tokens=True)
        
    L = len(token_ids)

    # 길이가 길면 그 문장을 넘어감.
    if L >= 50:
        return None
    
    # TOP5000에 없는 단어가 있는 문장이면 넘어감.
    for word in token_ids:
        try:
            Top5000_BtoA[word]
        except:
            return None
        
    return Top5000_BtoA[token_ids[1]]

In [0]:
counter = 0

first_pos = {}
first_neg = {}

length = len(pre_reviews)
for pre_review in pre_reviews:
  x = preprocess(pre_review[0], length)
  if x == None:
    continue
  try:
    if(pre_review[1]):
      first_pos[x] += 1
    else:
      first_neg[x] += 1
  except:
    if(pre_review[1]):
      first_pos[x] = 1
    else:
      first_neg[x] = 1

In [0]:
with gzip.open('/content/gdrive/My Drive/Top5000_AtoB.pickle','rb') as f:
    Top5000_AtoB = pickle.load(f)

In [0]:
first_pos_list = [(first_pos[key], tokenizer.decode(Top5000_AtoB[key])) for key in first_pos.keys()]
first_pos_list.sort(reverse = True)
first_neg_list = [(first_neg[key], tokenizer.decode(Top5000_AtoB[key])) for key in first_neg.keys()]
first_neg_list.sort(reverse = True)

In [0]:
first_pos_list = []
first_neg_list = []
acc = 0

for key in first_pos.keys():
  first_pos_list.append((key, acc))
  acc += first_pos[key]

first_pos_list.append((-1, acc))

acc = 0

for key in first_neg.keys():
  first_neg_list.append((key, acc))
  acc += first_neg[key]

first_neg_list.append((-1, acc))

import random

def random_number(first_list):
  r = random.randrange(first_list[-1][1])
  for i in range(len(first_list)):
    if(first_list[i][1] <= r and r < first_list[i+1][1]):
      return first_list[i][0]

In [0]:
with gzip.open('/content/gdrive/My Drive/Top5000_first_neg.pickle', 'wb') as f:
    pickle.dump(first_neg_list, f)

with gzip.open('/content/gdrive/My Drive/Top5000_first_pos.pickle', 'wb') as f:
    pickle.dump(first_pos_list, f)

In [0]:
import pickle
import gzip


with gzip.open('/content/gdrive/My Drive/Top5000_first.pickle','rb') as f:
    first_list = pickle.load(f)
    
with gzip.open('/content/gdrive/My Drive/Top5000_AtoB.pickle','rb') as f:
    AtoB = pickle.load(f)


In [0]:
real_first_list = []
for i in range(2889):
  x = first_list[i+1][1] - first_list[i][1]
  y = first_list[i][0]
  real_first_list.append((x, tokenizer.decode(AtoB[y])))